In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv('train (1).csv')
df_test = pd.read_csv('test (1).csv')

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
def ajustar_dados(df):
  df.drop(columns=['PoolQC','MiscFeature', 'Alley', 'Fence','MasVnrType','MasVnrType',
  'FireplaceQu',
  'GarageFinish',
  'GarageQual',
  'GarageCond',
  'GarageType',
  'GarageYrBlt',
  'BsmtExposure',
  'BsmtFinType2',
  'BsmtCond',
  'BsmtFinType1',
  'BsmtQual'],inplace=True)
  df.dropna(subset=['Electrical','MasVnrArea'],inplace=True)
  df['LotFrontage'].fillna(df['LotFrontage'].mean(),inplace=True)
  return df

df = ajustar_dados(df)
y = df['SalePrice']
# Align y with the cleaned dataframe (df)
y = y.loc[df.index]
df.drop(columns=['SalePrice','Id'],inplace=True)

/tmp/ipython-input-3851426270.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LotFrontage'].fillna(df['LotFrontage'].mean(),inplace=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical features that were not one-hot encoded
numerical_features = df.select_dtypes(exclude=['object']).columns

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply MinMaxScaler to the numerical features
df_new = pd.DataFrame(scaler.fit_transform(df[numerical_features]), columns=numerical_features)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Select object type columns
categorical_features = df_new.select_dtypes(include=['object']).columns

# Create a column transformer to apply OneHotEncoder to categorical features
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)],
    remainder='passthrough'
)

# Apply the transformation to the dataframe
df_encoded = ct.fit_transform(df_new)

# Convert the result back to a DataFrame (optional, but useful for inspection)
# Get feature names after transformation
all_feature_names = ct.get_feature_names_out()

df_encoded = pd.DataFrame(df_encoded, columns=all_feature_names)

display(df_encoded.head())

,remainder__MSSubClass,remainder__LotFrontage,remainder__LotArea,remainder__OverallQual,remainder__OverallCond,remainder__YearBuilt,remainder__YearRemodAdd,remainder__MasVnrArea,remainder__BsmtFinSF1,remainder__BsmtFinSF2,...,remainder__GarageArea,remainder__WoodDeckSF,remainder__OpenPorchSF,remainder__EnclosedPorch,remainder__3SsnPorch,remainder__ScreenPorch,remainder__PoolArea,remainder__MiscVal,remainder__MoSold,remainder__YrSold
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,...,0.386460,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,...,0.324401,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,...,0.428773,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,...,0.452750,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.090909,0.00
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,...,0.589563,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50


In [ ]:
df_encoded.describe()

,remainder__MSSubClass,remainder__LotFrontage,remainder__LotArea,remainder__OverallQual,remainder__OverallCond,remainder__YearBuilt,remainder__YearRemodAdd,remainder__MasVnrArea,remainder__BsmtFinSF1,remainder__BsmtFinSF2,...,remainder__GarageArea,remainder__WoodDeckSF,remainder__OpenPorchSF,remainder__EnclosedPorch,remainder__3SsnPorch,remainder__ScreenPorch,remainder__PoolArea,remainder__MiscVal,remainder__MoSold,remainder__YrSold
count,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,...,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000
mean,0.217254,0.167903,0.043038,0.565970,0.572450,0.718061,0.579336,0.064848,0.078362,0.031776,...,0.333234,0.110167,0.084867,0.039769,0.006753,0.031572,0.003762,0.002823,0.483554,0.453825
std,0.249120,0.075483,0.046708,0.153496,0.139177,0.218770,0.344189,0.113193,0.080682,0.109754,...,0.151038,0.146367,0.120768,0.110703,0.057888,0.116495,0.054609,0.032106,0.245487,0.332344
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.133562,0.029155,0.444444,0.500000,0.594203,0.266667,0.000000,0.000000,0.000000,...,0.230959,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.250000
50%,0.176471,0.167903,0.038220,0.555556,0.500000,0.724638,0.716667,0.000000,0.067505,0.000000,...,0.337094,0.000000,0.043876,0.000000,0.000000,0.000000,0.000000,0.000000,0.454545,0.500000
75%,0.294118,0.198630,0.048143,0.666667,0.625000,0.927536,0.900000,0.103750,0.125266,0.000000,...,0.406206,0.196033,0.124314,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selecionar as 6 melhores features usando ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=28)
X = selector.fit_transform(df_encoded, y)

mask = selector.get_support()
selected_features = df_encoded.columns[mask]


print("Features escolhidas:", list(selected_features))

Features escolhidas: ['remainder__LotFrontage', 'remainder__LotArea', 'remainder__OverallQual', 'remainder__OverallCond', 'remainder__YearBuilt', 'remainder__YearRemodAdd', 'remainder__MasVnrArea', 'remainder__BsmtFinSF1', 'remainder__BsmtFinSF2', 'remainder__BsmtUnfSF', 'remainder__TotalBsmtSF', 'remainder__1stFlrSF', 'remainder__2ndFlrSF', 'remainder__GrLivArea', 'remainder__BsmtFullBath', 'remainder__FullBath', 'remainder__HalfBath', 'remainder__BedroomAbvGr', 'remainder__TotRmsAbvGrd', 'remainder__Fireplaces', 'remainder__GarageCars', 'remainder__GarageArea', 'remainder__WoodDeckSF', 'remainder__OpenPorchSF', 'remainder__ScreenPorch', 'remainder__PoolArea', 'remainder__MiscVal', 'remainder__YrSold']


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

space = [
    # RandomForest
    Integer(50, 200, name="n_estimators_rf"),
    Integer(2, 15, name="max_depth_rf"),

    # SVR
    Real(0.1, 10.0, name="C_svm"),
    Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name="kernel_svm"),

    # XGBoost
    Integer(50, 200, name="n_estimators_xgb"),
    Real(0.01, 0.3, name="learning_rate_xgb"),
    Integer(2, 10, name="max_depth_xgb"),
]


# Função objetivo
@use_named_args(space)
def objective(**params):
    # Random Forest Regressor
    rf = RandomForestRegressor(
        n_estimators=params["n_estimators_rf"],
        max_depth=params["max_depth_rf"],
        random_state=42
    )

    # Suporte Vetorial para Regressão
    svr = SVR(
        C=params["C_svm"],
        kernel=params["kernel_svm"]
    )

    # XGBoost Regressor
    xgb = XGBRegressor(
        n_estimators=params["n_estimators_xgb"],
        learning_rate=params["learning_rate_xgb"],
        max_depth=params["max_depth_xgb"],
        eval_metric="rmse",
        random_state=42
    )

    # Meta-modelo: Regressão Linear
    meta_lr = LinearRegression()

    # Stacking (RF + SVR + XGB como base, LinearRegression como meta-modelo)
    stacking = StackingRegressor(
        estimators=[("rf", rf), ("svr", svr), ("xgb", xgb)],
        final_estimator=meta_lr,
        passthrough=True
    )

    # Voting (RF + SVR + XGB)
    voting = VotingRegressor(
        estimators=[("rf", rf), ("svr", svr), ("xgb", xgb)]
    )

    # Avaliar ambos
    scores_stack = cross_val_score(stacking, X, y, cv=5, scoring="neg_root_mean_squared_error")
    scores_vote = cross_val_score(voting, X, y, cv=5, scoring="neg_root_mean_squared_error")

    # Como o sklearn usa valores negativos para erros, invertemos o sinal
    mean_score = 0.5 * np.mean(scores_stack) + 0.5 * np.mean(scores_vote)

    return -mean_score  # minimizar RMSE

# Rodar otimização
res = gp_minimize(objective, space, n_calls=40, random_state=42)
print(res)

          fun: 32181.8397851704
            x: [np.int64(50), np.int64(10), 10.0, np.str_('poly'), np.int64(200), 0.12514484143243712, np.int64(2)]
    func_vals: [ 3.549e+04  3.448e+04 ...  3.218e+04  3.297e+04]
      x_iters: [[np.int64(169), np.int64(4), 7.818940902700418, 'rbf', np.int64(117), 0.03899272558722084, np.int64(6)], [np.int64(100), np.int64(4), 6.543795882193645, 'linear', np.int64(158), 0.2821802856145676, np.int64(2)], [np.int64(199), np.int64(10), 6.1553662888339815, 'linear', np.int64(53), 0.16218465147493288, np.int64(5)], [np.int64(57), np.int64(15), 2.4044362702600126, 'linear', np.int64(143), 0.12091397746747722, np.int64(10)], [np.int64(120), np.int64(13), 6.83504463201902, 'poly', np.int64(52), 0.2832385091486074, np.int64(7)], [np.int64(108), np.int64(2), 2.385848873659276, 'linear', np.int64(152), 0.1868990307569601, np.int64(9)], [np.int64(76), np.int64(7), 1.9041372691018175, 'sigmoid', np.int64(114), 0.07030308223177477, np.int64(7)], [np.int64(55), np.in

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos base
rf = RandomForestRegressor(n_estimators=50,max_depth=10, random_state=42)
gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.2, max_depth=2, random_state=42)
svr = SVR(C=10, kernel='poly')

# ----- Voting Regressor -----
voting_reg = VotingRegressor(estimators=[("rf", rf), ("gb", gb), ("svr", svr)])
voting_reg.fit(X_train, y_train)
y_pred_vote = voting_reg.predict(X_test)
print("Voting RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_vote)))

# ----- Stacking Regressor -----
stack_reg = StackingRegressor(
    estimators=[("rf", rf), ("gb", gb), ("svr", svr)],
    final_estimator=svr
)
stack_reg.fit(X_train, y_train)
y_pred_stack = stack_reg.predict(X_test)
print("Stacking RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_stack)))

Voting RMSE: 27788.65060929473
Stacking RMSE: 25926.19799031507


In [ ]:
df[df_test.isna()]

,0
MSSubClass,0
LotFrontage,0
LotArea,0
OverallQual,0
OverallCond,0
YearBuilt,0
YearRemodAdd,0
MasVnrArea,0
BsmtFinSF1,1
BsmtFinSF2,1


In [ ]:

submission_id = df_test['Id']
df_test.drop(columns=['Id'],inplace=True)
df_test = pd.DataFrame(scaler.transform(df_test[numerical_features]), columns=numerical_features)
df_test_encoded = pd.DataFrame(ct.transform(df_test), columns=all_feature_names)
X_test = df_test_encoded[selected_features]
#identifica o menor rmse e treina o conjunto de test para gerar a submission
y_pred_stack = stack_reg.predict(X_test)

KeyError: 'Id'

In [ ]:
submission = pd.DataFrame({'Id': submission_id, 'SalePrice': y_pred_stack})
submission.to_csv('submission.csv', index=False)